In [3]:
!ollama list

NAME                 ID              SIZE      MODIFIED       
llama3:latest        365c0bd3c000    4.7 GB    18 minutes ago    
deepseek-r1:7b       0a8c26691023    4.7 GB    26 hours ago      
deepseek-r1:7b-8k    d823319ccf43    4.7 GB    30 hours ago      


In [5]:
## time
from smolagents import OpenAIServerModel, CodeAgent, ToolCallingAgent, HfApiModel, tool, GradioUI
from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
import os

load_dotenv()

tool_model_id = os.getenv("TOOL_MODEL_ID")
reasoning_model_id = os.getenv("REASONING_MODEL_ID")
huggingface_api_token = os.getenv("HUGGINGFACE_API_TOKEN")

def get_model(model_id):
    using_huggingface = os.getenv("USE_HUGGINGFACE", "yes").lower() == "yes"
    if using_huggingface:
        return HfApiModel(model_id=model_id, token=huggingface_api_token)
    else:
        return OpenAIServerModel(
            model_id=model_id,
            api_base="http://localhost:11434/v1",
            api_key="ollama"
        )

# Create the reasoner for better RAG
reasoning_model = get_model(reasoning_model_id)
reasoner = CodeAgent(tools=[], model=reasoning_model, add_base_tools=False, max_steps=2)

# Initialize vector store and embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    model_kwargs={'device': 'cpu'}
)

#db_dir = os.path.join(os.path.dirname(__file__),"chroma_db")
db_dir = "chroma_db"
vectordb = Chroma(persist_directory=db_dir, embedding_function=embeddings)

@tool
def rag_with_reasoner(user_query: str) -> str:
    """
    This is a RAG tool that takes in a user query and searches for relevant content from the vector database.
    The result of the search is given to a reasoning LLM to generate a response, so what you'll get back
    from this tool is a short answer to the user's question based on RAG context.

    Args:
        user_query: The user's question to query the vector database with.
    """
    # Search for relevant documents
    docs = vectordb.similarity_search(user_query, k=3)
    
    # Combine document contents
    context = "\n\n".join(doc.page_content for doc in docs)
    
    # Create prompt with context
    prompt = f"""Based on the following context, answer the user's question. Be concise and specific.
    If there isn't sufficient information, give as your answer a better query to perform RAG with.
    
Context:
{context}

Question: {user_query}

Answer:"""
    
    # Get response from reasoning model
    response = reasoner.run(prompt, reset=False)
    return response

# Create the primary agent to direct the conversation
tool_model = get_model(tool_model_id)

agent = CodeAgent(tools=[rag_with_reasoner], model=tool_model, max_steps=4, verbosity_level=2)

agent_input = "Compare and contrast the services offered by RankBoost and Omni Marketing"
agent_output = agent.run(agent_input)
print("Final output:")
print(agent_output)


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Compare and contrast the services offered by RankBoost and Omni Marketing                                       │
│                                                                                                                 │
╰─ OpenAIServerModel - deepseek-r1:7b-8k ─────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
<think>                                                                                                            
Alright, I have to compare and contrast the services provided by RankBoost and Omni Marketing. Let me start by     
understanding what each company offers.                                                                            
                                                                                                                   
First, RankBoost is known for providing targeted online advertising solutions. They probably focus on Search Engine
Optimization (SEO) and Pay-Per-Click (PPC) campaigns because that's common in the digital marketing space. Their   
strength is likely in delivering precise ad placements to their target audience, which helps businesses attract    
more relevant visitors through their ads.                                                                          
                                                                                                                   
On the other hand, Omni Marketing seems broader in scope. They must offer a comprehensive suite of marketing       
services encompassing more channels than just SEO and PPC. This could include social media marketing, content      
creation like blogs or videos, web design for better user experience, event planning to engage potential customers 
at different stages, digital marketing strategies that integrate all these elements, and even loyalty programs to  
retain customers.                                                                                                  
                                                                                                                   
To effectively contrast them, I'll outline the key areas where each excels. RankBoost's focus on targeted ads      
likely gives them a niche expertise in driving specific desired actions through SEO and PPC. They probably have    
deep tools for tracking[38;2;230;237;2

Error in code parsing:
Your code snippet is invalid, because the regex pattern ```(?:py|python)?\n(.*?)\n``` was not found in it.
            Here is your code snippet:
            <think>
Alright, I have to compare and contrast the services provided by RankBoost and Omni Marketing. Let me start by 
understanding what each company offers.

First, RankBoost is known for providing targeted online advertising solutions. They probably focus on Search Engine
Optimization (SEO) and Pay-Per-Click (PPC) campaigns because that's common in the digital marketing space. Their 
strength is likely in delivering precise ad placements to their target audience, which helps businesses attract 
more relevant visitors through their ads.

On the other hand, Omni Marketing seems broader in scope. They must offer a comprehensive suite of marketing 
services encompassing more channels than just SEO and PPC. This could include social media marketing, content 
creation like blogs or videos, web design for better user experience, event planning to engage potential customers 
at different stages, digital marketing strategies that integrate all these elements, and even loyalty programs to 
retain customers.

To effectively contrast them, I'll outline the key areas where each excels. RankBoost's focus on targeted ads 
likely gives them a niche expertise in driving specific desired actions through SEO and PPC. They probably have 
deep tools for tracking conversions directly linked to their ad placements. However, they might lack broader 
marketing strategies that other companies offer.

Omni Marketing, by covering multiple channels, can handle a wider range of client needs, making it suitable for 
businesses with diverse targets or more complex marketing objectives. Their ability to integrate different aspects 
like content and events could provide a holistic customer experience. But since their services are spread across 
various channels, they might be less specialized than RankBoost in any single aspect.

Considering the future, as digital marketing grows, I can foresee that tools like RankBoost may expand their 
offerings into broader areas such as social media or content creation to match Omni Marketing's capabilities. 
However, for specific targeting and ad optimization, RankBoost might remain a key player.

In conclusion, while both offer significant value, the choice between them depends on whether a business needs 
specialized targeted ads (RankBoost) or a comprehensive marketing strategy across multiple platforms (Omni 
Marketing). Each has its strengths based on their service focus.
</think>

To compare and contrast the services offered by RankBoost and Omni Marketing, let's break down each service:

### **RankBoost Services**
- **Targeted Advertising Solutions**: RankBoost specializes in delivering precise online advertisements. Their 
services are likely focused on Search Engine Optimization (SEO) and Pay-Per-Click (PPC) campaigns.

  - This includes creating targeted ads to attract a specific audience, improving website traffic, and driving 
conversions.

### **Omni Marketing Services**
- **Comprehensive Marketing Suite**: Omni Marketing offers a wide array of marketing services. These may include:

  - **Social Media Marketing**: Engaging on platforms like Facebook, Instagram, etc., through targeted ads.
  - **Content Creation**: Providing blogs, videos, and other content to engage visitors better.
  - **Web Design**: Enhancing user experience with modern website design elements.

- **Integration of Multiple Channels**: The services integrate across various marketing channels. For instance:

  - Delivering a complete digital marketing strategy that includes SEO, social media ads; leveraging content 
through blogs or video; and managing events like webinars or workshops to engage potential clients.

### **Contrast Between RankBoost and Omni Marketing**
- | **Aspect**            | **RankBoost**                    | **Omni Marketing**
- |-------

[Step 1: Duration 43.16 seconds| Input tokens: 2,120 | Output tokens: 843]

...

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
<think>                                                                                                            
Alright, I've got this task where I need to compare and contrast the services provided by RankBoost and Omni       
Marketing. Let me start by understanding each company's offerings.                                                 
                                                                                                                   
**RankBoost**: From what I know, RankBoost is a company that specializes in targeted online advertising solutions. 
They likely focus on Search Engine Optimization (SEO) and Pay-Per-Click (PPC) campaigns. Their expertise probably  
lies in delivering precise ad placements to attract specific target audiences. This means they help businesses     
drive relevant traffic through high-conversion ads.                                                                
                                                                                                                   
**Omni Marketing**: On the other hand, Omni Marketing seems like a more comprehensive marketing firm. They offer a 
wide range of services across multiple channels. This could include digital marketing strategies that cover SEO and
PPC, as well as social media advertising. Beyond just ads, they might provide content creation, website design for 
better user engagement, event planning to engage potential customers, loyalty programs to retain clients, and so   
much more.                                                                                                         
                                                                                                                   
**Key Differences**:                                                                                               
- **Focus Area**: RankBoost is focused on targeted online advertising through SEO and PPC. Omni Marketing          
encompasses a broader range of marketing strategies.                                                               
                                                                                                                   
- **Specialization vs. Comprehensive Offering**: Rank Boost might[38;2;230;237;243;48;2;13;1

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Final Answer                                                                                                   
  final_answer("Omni Marketing")                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Omni Marketing

[Step 2: Duration 29.35 seconds| Input tokens: 6,043 | Output tokens: 1,442]

Final output:
Omni Marketing


In [7]:
## registry.ollama.ai/library/deepseek-r1:7b-8k does not support tools
primary_agent = ToolCallingAgent(tools=[rag_with_reasoner], model=tool_model, add_base_tools=False, max_steps=3)

agent_input = "Compare and contrast the services offered by RankBoost and Omni Marketing"
agent_output = primary_agent.run(agent_input)
print("Final output:")
print(agent_output)


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Compare and contrast the services offered by RankBoost and Omni Marketing                                       │
│                                                                                                                 │
╰─ OpenAIServerModel - deepseek-r1:7b-8k ─────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating tool call with model:
Error code: 400 - {'error': {'message': 'registry.ollama.ai/library/deepseek-r1:7b-8k does not support tools', 
'type': 'api_error', 'param': None, 'code': None}}

[Step 1: Duration 2.07 seconds| Input tokens: 3,923 | Output tokens: 599]

AgentGenerationError: Error in generating tool call with model:
Error code: 400 - {'error': {'message': 'registry.ollama.ai/library/deepseek-r1:7b-8k does not support tools', 'type': 'api_error', 'param': None, 'code': None}}